# 제1회 KRX 금융 빅데이터 활용 아이디어 경진대회

## 주제 : 개인투자자의 ktop30 감정지표

- __chapter 1. 크롤링__
    - 유튜브댓글 api
    - 네이버금융 종목토론방 댓글 크롤링
- __chapter 2. 데이터 전처리__
    - 댓글 전처리
    - 댓글 토크나이징
    - 댓글 레이블링
    - 학습데이터 생성 (댓글 도미넌스 기반 랜덤추출)
- __chapter 3. Bert 모델링__
- __chapter 4. 시각화 및 서비스__


- Reference
    - [한국거래소](http://www.krx.co.kr/main/main.jsp)
    - [네이버 금융](https://finance.naver.com/)
    - [CNN Fear & Greed Index](https://edition.cnn.com/markets/fear-and-greed) -> 정형지표만 이용한듯 해요
    - [Crypto Fear & Greed Index](https://alternative.me/crypto/fear-and-greed-index/)
    - 단어집 위치


---

## Abstract
    금융시장은 투자자의 시장인식에 따라 영향이 있을것이라 생각했습니다. 국내에서는 심리를 대변하는 지수로 정형지표의 계산으로 산출된 VIX지수를 지표로 이용합니다. 그러나 Crypto Fear & Greed Index 에서 비정형 데이터로 트위터 데이터를 산출식에 포함시키는 시도가 있었습니다. (가이드글)

    # 개인투자자의 시장인식이 댓글형태로 나타나는 네이버 종목토론실 댓글을 이용합니다. # 개인투자자의 시장인식을 수치화시켜 하나의 지표로 만들었습니다. # KODEX30 으로 프로토타입을 작성했습니다.

    인덱스가 FEAR(0)에 가까울수록 투자자가 느끼는 시장인식은 가격이 하락하고 있음을 대변합니다. 인덱스 활용방법으로는 매수시점을 정하는데 도움을 받을 수 있습니다. 반대의 경우 가격이 상승하는것을 대변하고, 매도시점을 정하는데 도움을 받을 수 있습니다.


- 사용데이터소개 (비정형/정형)
- 인덱스의 기대효과

---

# chapter 1. 크롤링

### 1.1 유튜브댓글 api
- 설명

### 1.2 종목토론실 댓글 크롤링
- 종목토론실 댓글에서 '날짜', '댓글제목', '조회수', '좋아요', 싫어요' 를 크롤링 했습니다.
- 기업대상은 KODEX30의 기업입니다.

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import datetime as dt


import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_code(symbol):
    krx = pd.read_csv('./krx_code.csv',encoding='utf-8')
    krx = krx.set_index('한글 종목약명')
    try:
        code = krx.at[symbol,'단축코드']
        return code
    except:
        print('종목명을 다시 확인해주세요.')
        return 0

def get_comment_csv(symbol,page,year,month,day):   
    code = get_code(symbol)
    date_list = [] # 날짜
    comment_list = [] # 댓글
    view_list = [] # 조회수
    good_list = [] # 좋아요
    bad_list = [] # 싫어요
    flag = 0
    for i in range(1,page+1):
        url = f'https://finance.naver.com/item/board.naver?code={code}&page={i}'
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50'}
        res = requests.get(url, headers = headers)
        bs = BeautifulSoup(res.text, 'html.parser')
        for j in range(20):
            try:
                root = bs.find('div',{'class':'section inner_sub'}).find_all('tr',{'onmouseover':'mouseOver(this)'})[j].text.split('\n')
                
                date_list.append(root[1].replace('.','-'))
                
                if len(root) == 14: # 답글
                    comment_list.append('답글:'+root[4])
                    view_list.append(root[10])
                    good_list.append(root[11])
                    bad_list.append(root[12])          
                elif len(root) == 13: # 기본
                    comment_list.append(root[3])
                    view_list.append(root[9])
                    good_list.append(root[10])
                    bad_list.append(root[11])
                else: # 에러
                    comment_list.append('error')
                    view_list.append(0)
                    good_list.append(0)
                    bad_list.append(0)   
            except:
                break
#                 date_list.append('error')
#                 comment_list.append('error')
#                 view_list.append(0)
#                 good_list.append(0)
#                 bad_list.append(0)  
            tp = [int(j) for j in root[1].split()[0].split('.')]
            if dt.datetime(tp[0],tp[1],tp[2]) < dt.datetime(year,month,day):
                flag = 1
                break
        if flag == 1:
            break
        print(f'\r{i}페이지 크롤링 완료.',end='')
        
#         for i in date_list:
#             tp = [int(j) for j in i.split()[0].split('-')]
#             if dt.datetime(tp[0],tp[1],tp[2]) < dt.datetime(year,month,day):
#                 flag = 1
#                 break
#         if flag == 1:
#             break
        
    df = pd.DataFrame()
    df['날짜'] = date_list
    df['댓글'] = comment_list
    df['조회수'] = view_list
    df['좋아요'] = good_list
    df['싫어요'] = bad_list
    return df

- ktop30 리스트 입니다.

In [ ]:
pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)

In [ ]:
ktop30_company = pd.read_excel('./KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)['종목명']

- 샘플 실행결과 입니다. 
- 인풋 파라미터는 기업이름,최대 페이지수,년,월,일 입니다. 설정한 년/월/일 이후의 댓글이 나오면 크롤링을 멈추게 됩니다. 

In [ ]:
df = get_comment_csv('미래에셋증권',500,2022,6,1) # sample
df

In [ ]:
# 실행 건너뛰기
for i in range(3):
    df = get_comment_csv(list(ktop30_company)[i],100000,2020,6,1)
    #df = df[df['날짜']=='2022-07-01']
    df.to_csv(f'./src/2year_time/{list(ktop30_company)[i]}_2year.csv')

In [ ]:
# 크롤링된 데이터 확인
pd.read_csv('./src/year_time/삼성전자_year.csv')

# chapter 2. 데이터 전처리

### 2.1 댓글 전처리
- 온전한 한글문자 이외에 모든 부분을 제거했습니다.
- 초성,알파벳,특수문자,이모티콘 등이 제거 되었습니다.


- 예시 코드입니다.

In [ ]:
import pandas as pd

In [ ]:
target = pd.read_csv('./src/2year_time/삼성화재_2year.csv')
target

In [ ]:
def clean_sents_df(company):
    try:
        target = pd.read_csv(f'./src/{company}_2year.csv',encoding='utf8').drop('Unnamed: 0',axis=1)
    except:
        target = pd.read_csv(f'./src/{company}_2year.csv',encoding='utf8')

    if company == 'sampro':
        target.rename(columns={'comment':'댓글'},inplace=True)
    df = target
    df['정제된 댓글'] = df['댓글'].str.replace('\\[삭제된 게시물의 답글\\]',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('답글:',' ')
    df['정제된 댓글'] = df['정제된 댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip()
    df = df[df['정제된 댓글'] != '']
    df = df.reset_index(drop=True)
    return  df

### 2.2 댓글 토크나이징

- Bert 모델링을 위한 labeling 사전작업 입니다.

In [ ]:
def corpus_save(company):
    df = clean_sents_df(company)
    df['정제된 댓글 길이'] = [len(str(i)) for i in df['정제된 댓글']]
    df = df[df['정제된 댓글 길이'] > 5]

    tp = [str(i) for i in list(df['정제된 댓글'])]
    save = '\n'.join(tp)
    f = open("./corpus_target.txt", 'a',encoding='utf8')
    f.write(save)
    f.close()
    
def corpus_init():
    ktop30_company = pd.read_excel('./src/KODEX_KTOP_30_20220629.xlsx',header=2).drop(0,axis=0)['종목명']
    company_set = list(ktop30_company)
    company_set.append('sampro')
    f = open("./corpus_target.txt", 'w',encoding='utf8')
    f.write('')
    f.close()
    for company in company_set:
        corpus_save(company)

def return_tokenizer():
    corpus = DoublespaceLineCorpus("./src/corpus_target.txt",iter_sent=True)
    noun_extractor = LRNounExtractor_v2(verbose=True)
    nouns = noun_extractor.train_extract(corpus)
    scores = {word:score.score for word, score in nouns.items()}
    tokenizer = LTokenizer(scores=scores)
    return tokenizer

In [ ]:
corpus_init()
tokenizer = return_tokenizer()

- 3181878 개의 네이버 종목토론방, 유튜브 댓글을 학습시키고, 학습된 tokenizer를 이용하여 전처리 댓글을 토큰화 시킵니다.

In [ ]:
target_df = clean_sents_df('삼성전자')
target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
target_df

### 2.3 댓글 레이블링
- fear_words_set 과 greed_words_set을 설정하고 토크나이징댓글의 요소가 fear 단어집에 있으면 -1씩 greed 단어집에 있으면 +1씩 부여했습니다.
- 양수인경우 greed, 음수인경우 fear로 labeling 했습니다. 0 인경우 일단 데이터프레임에 기록되고, 최종적으로 train_data 선정시에는 제외됩니다.

In [ ]:
def labeling(target_df):
    
    f = open("./neg_pol_word.txt", 'r',encoding='utf8')
    words = f.readlines()
    f.close()
    fear_words_set = {word.strip('\n') for word in words}

    f = open("./pos_pol_word.txt", 'r',encoding='utf8')
    words = f.readlines()
    f.close()
    greed_words_set = {word.strip('\n') for word in words}
    
    label_score = []
    for token_list in target_df['토큰화 댓글']:
        sent_score = 0
        for token in token_list:
            if token in fear_words_set:
                sent_score -= 1
            elif token in greed_words_set:
                sent_score += 1

        if sent_score < 0:
            label_score.append(-1)
        elif sent_score > 0:
            label_score.append(1)
        else:
            label_score.append(0)
            
    target_df['label'] = label_score
    
    return target_df

### 2.4 학습데이터 생성

- 학습데이터는 종목토론실 댓글에서 3만개, 유튜브 댓글에서 1만개 랜덤추출 했습니다.

In [ ]:
def setting_train_data():
    company = list(pd.read_excel('./KODEX_KTOP_30_20220715.xls',header=2)['종목명'][1:])
    dominance = list(pd.read_excel('./KODEX_KTOP_30_20220715.xls',header=2)['비중(%)'][1:])

    train_data = pd.DataFrame(columns=['날짜','댓글','조회수','좋아요','싫어요','정제된 댓글','토큰화 댓글','label'])
    for idx in range(30):
        target_df = clean_sents_df(company[idx])
        target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
        
        label_df = labeling(target_df)
        
        label_df = label_df[label_df['label'] != 0] # except label:0 
        label_df = label_df.sample(int(30000*dominance[idx])+1,replace=True) # replace option : 샘플수가 부족할경우 중복이 생김 ** 추후 수정
        train_data = train_data.append(label_df)
    
    return train_data

In [ ]:
train_data = setting_train_data()
train_data.reset_index(inplace=True)
train_data.drop('index',axis=1,inplace=True)
train_data

In [ ]:
# 삼프로
target_df = clean_sents_df('sampro')
target_df['토큰화 댓글'] = [tokenizer(str(i)) for i in target_df['정제된 댓글']]
label_df = labeling(target_df)

label_df = label_df[label_df['label'] != 0] # except label:0 
label_df = label_df.sample(10000,replace=True) # replace option : 샘플수가 부족할경우 중복이 생김 ** 추후 수정
train_data = train_data.append(label_df)

train_data = train_data.loc[:,['정제된 댓글','label']].reset_index().drop('index',axis=1)

train_data

# chapter 3. Bert 모델링

# chapter 4. 시각화 및 서비스